# Extarct handcrafted features 
- for each person, for x frames, calculate the nonverbal features from head and body landmarks 
- the x frames are: 64 frames (\~2.5s), 64\*2 (\~5s) and 5*30(=5s)
- the first two are to be compared with the deep features, the last is for the actual labeling 

- raw data --> low-level --> high-level 

### imports and vars

In [ ]:
import os
import sys
import numpy as np
import pandas as pd 


raw_features_path = '/home/sharifa/speedDating/speedDating_Detectron_named_3D_with_gazeFollow/'


### help functions 

In [ ]:
MidHip = 0
RHip = 1
RKnee = 2
RAnkle = 3
LHip = 4
LKnee = 5
LAnkle = 6
MidBack = 7
Neck = 8
Nose = 9
forehead =10
LShoulder = 11
LElbow = 12
LWrist = 13
RShoulder = 14
RElbow = 15
RWrist = 16

def body_get_headers():
    body_features_heads =['parent_body_pitch', 'parent_body_roll', 'parent_body_yaw',
                          'child_body_pitch', 'child_body_roll', 'child_body_yaw',
                          'both_body_pitch', 'both_body_roll', 'both_body_yaw',
                         'both_body_dist', 'childlhand_childface', 'childlhand_parentface', 'parentlhand_childface',
                         'parentlhand_parentface', 'childrhand_childface', 'childrhand_parentface', 'parentrhand_childface',
                         'parentrhand_parentface', 'childlhand_childbody', 'childlhand_parentbody', 'parentlhand_childbody',
                         'parentlhand_parentbody', 'childrhand_childbody', 'childrhand_parentbody', 'parentrhand_childbody',
                         'parentrhand_parentbody', 'childlhand_childrhand', 'parentlhand_parentrhand', 'childlhand_parentlhand',
                         'childlhand_parentrhand', 'childrhand_parentlhand', 'childrhand_parentrhand']
    return body_features_heads



def head_get_headers():
    head_features_heads = ['parent_pitch', 'parent_roll', 'parent_yaw', 'child_pitch', 'child_roll', 'child_yaw', 'head_distances', 'gazes_pitch', 'gazes_roll', 'gazes_yaw']

    return head_features_heads

def get_stat_headers(features_heads):
    # return stat_features
    stats_features = ['f_min', 'f_max', 'f_rang', 'f_mean', 'f_std', 'f_var', 'f_skew', 'f_kurt', 'f_peaks', 'f_valys',
                         'd1_min', 'd1_max', 'd1_rang', 'd1_mean', 'd1_std', 'd1_var', 'd1_skew', 'd1_kurt', 'd1_peaks', 'd1_valys',
                         'd2_min', 'd2_max', 'd2_rang', 'd2_mean', 'd2_std', 'd2_var', 'd2_skew', 'd2_kurt', 'd2_peaks', 'd2_valys']

    high_feature_headers = []
    for i in range(len(features_heads)):
        for j in range(len(stats_features)):
            high_feature_headers.append(features_heads[i] + '-' + stats_features[j])

    return high_feature_headers



def chest_pose(chest_points, size):
    # Camera internals
    focal_length = size[1]
    center = (size[1] / 2, size[0] / 2)
    camera_matrix = np.array(
        [[focal_length, 0, center[0]],
         [0, focal_length, center[1]],
         [0, 0, 1]], dtype="double"
    )

    dist_coeffs = np.zeros((4, 1))  # Assuming no lens distortion
    d2_points = np.float64(chest_points[:, 0:2])

    (success, rotation_vector, translation_vector) = cv2.solvePnP(chest_points, d2_points, camera_matrix, dist_coeffs)
    # flags=cv2.cv2.SOLVEPNP_ITERATIVE)

    return rotation_vector


def head_pose(face_3d, size):

    # Camera internals
    focal_length = size[1]
    center = (size[1] / 2, size[0] / 2)
    camera_matrix = np.array(
        [[focal_length, 0, center[0]],
         [0, focal_length, center[1]],
         [0, 0, 1]], dtype="double"
    )

    dist_coeffs = np.zeros((4, 1))  # Assuming no lens distortion
    d2_points = np.float64(face_3d[:, 0:2])
    rvec = np.zeros(3, dtype=np.float)
    tvec = np.array([0, 0, 1], dtype=np.float)

    (success, rotation_vector, translation_vector) =\
        cv2.solvePnP(face_3d, d2_points, camera_matrix, dist_coeffs,\
                     rvec, tvec, useExtrinsicGuess=True, flags=cv2.SOLVEPNP_ITERATIVE)

    return rotation_vector


def body_sync(parent_body, child_body, frame_size):
    body_features = []

    if len(parent_body) == 0 or len(child_body) == 0:
        #print("no parent and/or child were detected")
        return body_features

    parent_chest = np.array([parent_body[Neck], parent_body[RShoulder], parent_body[LShoulder], parent_body[MidHip]])
    child_chest = np.array([child_body[Neck], child_body[RShoulder], child_body[LShoulder], child_body[MidHip]])

    p_rot = chest_pose(parent_chest, frame_size)
    pb_pitch, pb_roll, pb_yaw = p_rot[0][0], p_rot[1][0], p_rot[1][0]

    c_rot = chest_pose(child_body, frame_size)
    cb_pitch, cb_roll, cb_yaw = c_rot[0][0], c_rot[1][0], c_rot[1][0]

    ret_R, ret_t = rigid_transform_3D(np.mat(parent_chest), np.mat(child_chest))

    b_pitch, b_roll, b_yaw = 0, 0, 0
    euler_angles = euler_from_matrix(ret_R)
    if euler_angles:
        #print('THEY SHOULD BE LOOKING AT EACH OTHER')
        b_pitch, b_roll, b_yaw = euler_angles[0], euler_angles[1], euler_angles[2]

    p_nose, p_center, p_rhand, p_lhand = None, None, None, None
    c_nose, c_center, c_rhand, c_lhand = None, None, None, None

    # if parent_body[0][0] != 0.0 and parent_body[0][1] != 0.0 and parent_body[0][2] != 0.0:
    p_nose = parent_body[Nose]
    # if parent_body[1][0] != 0.0 and parent_body[1][1] != 0.0 and parent_body[1][2] != 0.0:
    p_center = parent_body[Neck]
    # if parent_body[4][0] != 0.0 and parent_body[4][1] != 0.0 and parent_body[4][2] != 0.0:
    p_rhand = parent_body[RWrist]
    # if parent_body[7][0] != 0.0 and parent_body[7][1] != 0.0 and parent_body[7][2] != 0.0:
    p_lhand = parent_body[LWrist]

    # if child_body[0][0] != 0.0 and child_body[0][1] != 0.0 and child_body[0][2] != 0.0:
    c_nose = child_body[Nose]
    # if child_body[1][0] != 0.0 and child_body[1][1] != 0.0 and child_body[1][2] != 0.0:
    c_center = child_body[Neck]
    # if child_body[4][0] != 0.0 and child_body[4][1] != 0.0 and child_body[4][2] != 0.0:
    c_rhand = child_body[RWrist]
    # if child_body[7][0] != 0.0 and child_body[7][1] != 0.0 and child_body[7][2] != 0.0:
    c_lhand = child_body[LWrist]

    b_dist = None
    parentlhand_parentface, parentrhand_parentface, childlhand_parentface, childrhand_parentface = None, None, None, None
    parentlhand_childface, parentrhand_childface, childlhand_childface, childrhand_childface = None, None, None, None
    parentlhand_parentbody, parentrhand_parentbody, childlhand_parentbody, childrhand_parentbody = None, None, None, None
    parentlhand_childbody, parentrhand_childbody, childlhand_childbody, childrhand_childbody = None, None, None, None
    childlhand_childrhand, parentlhand_parentrhand, childlhand_parentlhand, childlhand_parentrhand, childrhand_parentlhand, childrhand_parentrhand = None, None, None, None, None, None

    if type(p_center) != type(None) and type(c_center) != type(None):
        b_dist = np.linalg.norm(p_center - c_center)

    if type(p_nose) != type(None):
        if type(p_lhand) != type(None):
            parentlhand_parentface = np.linalg.norm(p_lhand - p_nose)
        if type(p_rhand) != type(None):
            parentrhand_parentface = np.linalg.norm(p_rhand - p_nose)
        if type(c_lhand) != type(None):
            childlhand_parentface = np.linalg.norm(c_lhand - p_nose)
        if type(c_rhand) != type(None):
            childrhand_parentface = np.linalg.norm(c_rhand - p_nose)

    if type(c_nose) != type(None):
        if type(p_lhand) != type(None):
            parentlhand_childface = np.linalg.norm(p_lhand - c_nose)
        if type(p_rhand) != type(None):
            parentrhand_childface = np.linalg.norm(p_rhand - c_nose)
        if type(c_lhand) != type(None):
            childlhand_childface = np.linalg.norm(c_lhand - c_nose)
        if type(c_rhand) != type(None):
            childrhand_childface = np.linalg.norm(c_rhand - c_nose)

    if type(p_center) != type(None):
        if type(p_lhand) != type(None):
            parentlhand_parentbody = np.linalg.norm(p_lhand - p_center)
        if type(p_rhand) != type(None):
            parentrhand_parentbody = np.linalg.norm(p_rhand - p_center)
        if type(c_lhand) != type(None):
            childlhand_parentbody = np.linalg.norm(c_lhand - p_center)
        if type(c_rhand) != type(None):
            childrhand_parentbody = np.linalg.norm(c_rhand - p_center)

    if type(c_center) != type(None):
        if type(p_lhand) != type(None):
            parentlhand_childbody = np.linalg.norm(p_lhand - c_center)
        if type(p_rhand) != type(None):
            parentrhand_childbody = np.linalg.norm(p_rhand - c_center)
        if type(c_lhand) != type(None):
            childlhand_childbody = np.linalg.norm(c_lhand - c_center)
        if type(c_rhand) != type(None):
            childrhand_childbody = np.linalg.norm(c_rhand - c_center)

    if type(p_lhand) != type(None):
        if type(p_rhand) != type(None):
            parentlhand_parentrhand = np.linalg.norm(p_lhand - p_rhand)
        if type(c_lhand) != type(None):
            childlhand_parentlhand = np.linalg.norm(c_lhand - p_lhand)
        if type(c_rhand) != type(None):
            childrhand_parentlhand = np.linalg.norm(c_rhand - p_lhand)

    if type(c_lhand) != type(None):
        if type(p_rhand) != type(None):
            childlhand_parentrhand = np.linalg.norm(c_lhand - p_rhand)
        if type(c_rhand) != type(None):
            childlhand_childrhand = np.linalg.norm(c_lhand - c_rhand)

    if type(p_rhand) != type(None) and type(c_rhand) != type(None):
        childrhand_parentrhand = np.linalg.norm(c_rhand - p_rhand)

    body_features = [pb_pitch, pb_roll, pb_yaw, cb_pitch, cb_roll, cb_yaw, b_pitch, b_roll, b_yaw,
                     b_dist, childlhand_childface, childlhand_parentface, parentlhand_childface,
                     parentlhand_parentface, childrhand_childface, childrhand_parentface, parentrhand_childface,
                     parentrhand_parentface, childlhand_childbody, childlhand_parentbody, parentlhand_childbody,
                     parentlhand_parentbody, childrhand_childbody, childrhand_parentbody, parentrhand_childbody,
                     parentrhand_parentbody, childlhand_childrhand, parentlhand_parentrhand, childlhand_parentlhand,
                     childlhand_parentrhand, childrhand_parentlhand, childrhand_parentrhand]

    return body_features


def head_sync(parent_body, child_body, frame_size):
    head_features = []

    parent_face = np.array([parent_body[forehead],parent_body[Nose],parent_body[Neck]])
    child_face = np.array([child_body[forehead],child_body[Nose],child_body[Neck]])

    if len(parent_face) == 0 or len(child_face) == 0:
        # print("no parent or child were detected")
        return head_features
    # print(pose_3d.shape)

    # set the parent face and the child face based on location

    nose = 1
    x_axes = 0

    # print(parent_face, child_face)

    # extract feature: pitch, roll, yaw for each head
    # parent head relative to origin
    p_rot = head_pose(parent_face, frame_size)
    p_pitch, p_roll, p_yaw = p_rot[0], p_rot[1], p_rot[1]
    # child head relative to origin
    c_rot = head_pose(child_face, frame_size)
    c_pitch, c_roll, c_yaw = c_rot[0], c_rot[1], c_rot[1]

    # extract distances between the two heads
    dist = np.linalg.norm(parent_face[nose] - child_face[nose])

    # print(dist)

    # extract gaze rotation between the two heads
    ret_R, ret_t = rigid_transform_3D(np.mat(parent_face), np.mat(child_face))

    euler_angles = euler_from_matrix(ret_R)

    g_pitch, g_roll, g_yaw = 0, 0, 0
    if euler_angles:
        g_pitch, g_roll, g_yaw = euler_angles[0], euler_angles[1], euler_angles[2]

    head_features = [p_pitch, p_roll, p_yaw, c_pitch, c_roll, c_yaw, dist, g_pitch, g_roll, g_yaw]
    return head_features


def extract_raw(family, sess, video_file, csv_path, csv_file):
    points_folder = os.path.join(conf.points_path_3D, family, sess, '{}_{}_mainvid.mp4'.format(family, sess),
                                 '{}_{}_mainvid_frame-json'.format(family, sess))
    files_total = len(os.listdir(points_folder))
    all_raw_body = []
    all_raw_head = []
    # keep looping infinitely
    # for i in tqdm(count()):
    for i in tqdm(range(files_total)):
        # if i >=100:
        #     break
        point_file = os.path.join(points_folder, '{}.npy'.format(str(i).zfill(6)))

        if not os.path.exists(point_file):
            all_raw_body.append([])
            all_raw_head.append([])
            continue
        im_res = np.load(point_file, allow_pickle=True)
        #plot BBox and name
        parent_body, child_body = [], []
        for human in im_res:
            # body joints
            joint3D_item = human['3d_keypoints']
            if human['face_name'] == 'parent':
                parent_body = joint3D_item
            elif human['face_name'] == 'child':
                child_body = joint3D_item

        if len(parent_body) ==0 or len(child_body) == 0:
            all_raw_body.append([])
            all_raw_head.append([])
            continue

        #sync features
        frame_size = (640 , 480)
        raw_body = body_sync(parent_body, child_body, frame_size)
        raw_head = head_sync(parent_body, child_body, frame_size)

        all_raw_body.append(raw_body)
        all_raw_head.append(raw_head)

    #save the csv with header
    import pandas as pd
    df = pd.DataFrame(all_raw_body)
    df.columns = body_get_headers()
    csv_file_body = 'Body_'+csv_file
    df.to_csv(os.path.join(csv_path,csv_file_body),header=True,index=False)

    #head file
    df = pd.DataFrame(all_raw_head)
    df.columns = head_get_headers()
    csv_file_head = 'Head_'+csv_file
    df.to_csv(os.path.join(csv_path,csv_file_head),header=True,index=False)


### step 1 -- extract and save low-level features (raw data --> low-level)
- raw data are saved per frame for every one in the frame
- wants to make low-level files one for each person
- coloumns = low-level features
- raws = each frame 


In [ ]:

    read_folder = conf.video_path
    family_folders = [f for f in os.listdir(read_folder) if f.startswith('p')]
    family_folders.sort()

    for family in family_folders:
        family_fol_read = read_folder + family + '/'
        session_folders = [f for f in os.listdir(family_fol_read) if f.startswith('s')]
        session_folders.sort()
        for sess in session_folders:
            sess_fol_read = family_fol_read + sess + '/'
            onlyvideofiles = [os.path.join(sess_fol_read, f) for f in os.listdir(sess_fol_read) if
                              os.path.isfile(os.path.join(sess_fol_read, f)) and f.endswith('mainvid.mp4')]

            for video in onlyvideofiles:
                read_video_file = video
                print('Processing:',read_video_file)

                csv_path = '/home/sharifa/Family_dataset_features/raw_features/'
                csv_file = os.path.basename(read_video_file).split('.')[0] + '.csv'

                if os.path.exists(os.path.join(csv_path,'Body_'+csv_file)) and \
                        os.path.exists(os.path.join(csv_path, 'Head_' + csv_file)):
                    print('Already processed.')
                    continue

                if family == 'p19' and sess == 's1':
                    print('Skipping')
                    continue
                extract_raw(family, sess, read_video_file, csv_path, csv_file)


### step 2 -- extract and save high-level features (low-level --> high-level)
- low-level features are merged for every x frames (explained above)
- temporal features are extarcted for that window size and saved per person 

- coloumns = high-level features
- raws = temporal features every x frame 
- saved in 3 variations:  64 frames (~2.5s), 64*2 (~5s) and 5*30(=5s)


In [ ]:

def statstic_features(features):
    features = np.array(features)
    # stats = np.zeros((1,NUM_BODY_STATS))
    epsilon = np.float32(0.00001)
    #for each feature/column
    for ff in tqdm(range(features.shape[1])):
        this_feature = features[:,ff]

        #find and remove outliers
        this_feature = grubbs.test(this_feature, alpha=0.05)
        #de-noise: smooth the signal - median filter
        this_feature = medfilt(this_feature)

        #extract the deltas
        delta1_this_feature = np.append(this_feature[0], np.diff(this_feature))
        delta2_this_feature = np.append(delta1_this_feature[0], np.diff(delta1_this_feature))

        # extract stats
        f_min = min(this_feature)
        d1_min = min(delta1_this_feature)
        d2_min = min(delta2_this_feature)

        f_max = max(this_feature)
        d1_max = max(delta1_this_feature)
        d2_max = max(delta2_this_feature)

        f_rang = f_max - f_min
        d1_rang = d1_max - d1_min
        d2_rang = d2_max - d2_min

        f_mean = mean(this_feature)
        try:
            d1_mean = mean(delta1_this_feature)
            d2_mean = mean(delta2_this_feature)
        except:
            print("SAD")
            print(delta1_this_feature)

        f_std = stdev(this_feature)
        d1_std = stdev(delta1_this_feature)
        d2_std = stdev(delta2_this_feature)

        f_var = variance(this_feature)
        d1_var = variance(delta1_this_feature)
        d2_var= variance(delta2_this_feature)

        #extarct skewness and kurtosis
        f_skew = skew(this_feature)
        d1_skew = skew(delta1_this_feature)
        d2_skew= skew(delta2_this_feature)

        f_kurt = kurtosis(this_feature)
        d1_kurt = kurtosis(delta1_this_feature)
        d2_kurt= kurtosis(delta2_this_feature)

        #extract peaks and valys

        try:
            test = (1 /delta1_this_feature)
        except:
            for i in range(len(delta1_this_feature)):
                if delta1_this_feature[i] == 0.0:
                    delta1_this_feature[i] = epsilon


        f_peaks = len(find_peaks(this_feature)[0])
        d1_peaks = len(find_peaks(delta1_this_feature)[0])
        d2_peaks = len(find_peaks(delta2_this_feature)[0])

        f_valys = len(find_peaks(1 /this_feature)[0])
        d1_valys = len(find_peaks(1 /delta1_this_feature)[0])
        d2_valys = len(find_peaks(1 /delta2_this_feature)[0])

        f_stats = np.array([f_min, f_max, f_rang, f_mean, f_std, f_var, f_skew, f_kurt, f_peaks, f_valys,
                     d1_min, d1_max, d1_rang, d1_mean, d1_std, d1_var, d1_skew, d1_kurt, d1_peaks, d1_valys,
                     d2_min, d2_max, d2_rang, d2_mean, d2_std, d2_var, d2_skew, d2_kurt, d2_peaks, d2_valys])

        if ff == 0:
            stats = f_stats
        else:
            stats = np.append(stats, f_stats, axis=0)
    return stats


-----
    read_folder = conf.compress_video_folder
    families = ['F' + str(i) for i in range(1, 18)]
    exclude_one_person = ['F' + str(i) for i in [9, 12, 14, 15, 16]]

    for family in families:
        if family in exclude_one_person:
            print('Skipping Interaction .. {}'.format(family))
            continue

        onlyvideofiles = [os.path.join(read_folder, f) for f in os.listdir(read_folder) if
                          os.path.isfile(os.path.join(read_folder, f)) and f.endswith('.mp4') and f.startswith(
                              family + '_')]

        for video in onlyvideofiles:
            read_video_file = video
            save_video_folder = os.path.basename(video).split('.')[0]

    read_folder = conf.video_path
    family_folders = [f for f in os.listdir(read_folder) if f.startswith('p')]
    family_folders.sort()

    id_header = ['FamilyID','Session']  + body_get_headers()
    all_high_body = pd.DataFrame(columns=id_header)

    id_header = ['FamilyID','Session']  + head_get_headers()
    all_high_head = pd.DataFrame(columns=id_header)

    for family in family_folders:
        family_fol_read = read_folder + family + '/'
        session_folders = [f for f in os.listdir(family_fol_read) if f.startswith('s')]
        session_folders.sort()
        for sess in session_folders:
            sess_fol_read = family_fol_read + sess + '/'
            onlyvideofiles = [os.path.join(sess_fol_read, f) for f in os.listdir(sess_fol_read) if
                              os.path.isfile(os.path.join(sess_fol_read, f)) and f.endswith('mainvid.mp4')]

            for video in onlyvideofiles:
                read_video_file = video
                print('Processing:',read_video_file)

                low_csv_path = '/home/sharifa/Family_dataset_features/raw_features/'
                high_csv_path = '/home/sharifa/Family_dataset_features/high_features/'
                low_csv_file = os.path.basename(read_video_file).split('.')[0] + '.csv'

                body_csv_file = os.path.join(low_csv_path,'Body_'+low_csv_file)
                head_csv_file = os.path.join(low_csv_path,'Head_'+low_csv_file)
                if os.path.exists(body_csv_file):
                    body_low_data = pd.read_csv(body_csv_file)
                    head_low_data = pd.read_csv(head_csv_file)
                else:
                    print('Empty')
                    continue

                this_body = statstic_features(body_low_data)
                this_head = statstic_features(head_low_data)

                this_id = np.array([family, sess])

                all_high_body = all_high_body.append(pd.Series(np.concatenate((this_id,this_body)),\
                                                               index=all_high_body.columns), ignore_index=True)
                all_high_head = all_high_head.append(pd.Series(np.concatenate((this_id,this_head)),\
                                                               index=all_high_head.columns),ignore_index=True)

    # print(all_high_body.head(10))
    # print(all_high_head.head(10))
    all_high_body.to_csv(os.path.join(high_csv_path,'all_high_body.csv'))
    all_high_head.to_csv(os.path.join(high_csv_path,'all_high_head.csv'))
    # split: body_parent, body_child, body_sync
    # splid array, header and save each
    # split: head_parent, head_child, head_sync
